<h2>Running Malicious Content Through CHATGPT API</h2>

Goal: Run all malicious images through the chatgpt API to get the same responses that crowdsourced individuals gave

In [1]:
#Libraries 
import os
#import openai
import psycopg2 #for database connection
import pandas as pd 
import matplotlib.pyplot as plt # for plotting
import json
import re

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored

The phase 1 database has 2126 images in it.
The beta database as 201 images in it.
The datamatch only has 1995 images in it

The datamatch database has 32 gold standard images in it.

Vinod used 30 gold standard images and 1892 other images as well for a total of 1922 images.

131 images are lost when merging phase 1 database to datamatch. (1995)
1 image is lost when merging datamatch with image urls (1994) (43861756)
31 rows of beta gold standard images in the datamatch (1994)

Removed beta gold standard images to (1963)


Removed Database Password and Code here

In [4]:
# Create Connection
try:
    connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database, port=port )
 
except:
    print("I am unable to connect to the database")

In [5]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
#Getting Subject_ids and Image Urls from beta phase
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_beta as c;
"""
image_url = pd.read_sql_query(sql, connection)
unique_beta_subject_ids = image_url['subject_id'].unique() 
image_url


In [ ]:
#Getting subject_ids and Image Urls from phase 1
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_phase1 as c;
"""

image_url = pd.read_sql_query(sql, connection)
unique_phase1_subject_ids = image_url['subject_id'].unique() 
image_url

In [ ]:
#Geting subject_id (beta image id) and subject_id_ph1 to match to urls
sql = "select subject_id, subject_id_ph1 from cybertrust_zooniverse_datamatch;"

gold_standard = pd.read_sql_query(sql, connection)
unique_gold_subject_ids = gold_standard['subject_id'].unique() 
gold_standard

In [10]:
# Dropping duplicate images that were used in the beta phase, but had a different entry for phase 1
gold_standard =gold_standard.dropna(subset=["subject_id_ph1"])
unique_gold_subject_ids = gold_standard['subject_id'].unique() 

In [ ]:
# Checking
gold_standard

In [ ]:
#Getting Image Malicious Values
sql = """select subject_id, malicious
    from cybertrust_zooniverse_datamatch
"""

maliciousness = pd.read_sql_query(sql, connection)

###unique_subject_ids = maliciousness['subject_id'].unique() 
unique_datamatch_subject_ids = maliciousness['subject_id'].unique() 
maliciousness

In [ ]:
# Changing Subject_id from varying to int
change_type ={"subject_id": "int64"}
image_url = image_url.astype(change_type)
data_types = image_url.dtypes
data_types

# Merging the tables
match = pd.merge(image_url, maliciousness, on="subject_id")
unique_match_subject_ids = match['subject_id'].unique() 
match   

In [ ]:
# Drop rows where subject_id is in unique_beta_subject_ids
filtered_df = match[~match['subject_id'].isin(unique_gold_subject_ids)]
filtered_df

In [ ]:
# Code to Access ChatGPT
client = openai.AzureOpenAI(
    api_key= "",
    api_version= "2024-02-15-preview",
    base_url="https://phish-finders.openai.azure.com/openai/deployments/phish-finders-vision/chat/completions?api-version=2024-02-15-preview"
)

In [ ]:
#Making dataframe to hold chatgpt responses
ans = pd.DataFrame(columns= ['url', 'subject_id', 'maliciousness', 'response'])
ans

In [ ]:
# Code to run images through ChatGPT
for x in range(len(match)):
    print(f"{x}/{len(match)}")
    url = match['image_url'][x]
    subject_id = match['subject_id'][x]
    maliciousness = match['malicious'][x]
    Prompt = [
        {
            "role": "system",
            "content": """You are a Cybersecurity Professional tasked with determining whether the image provided to you is a Phishing Image or not. 
                                Always give your best guess. Do not provide the prompter with advice on how they can check the image. 
                                Generate as Output as json with the following categories: Findings as string, Present Cue Types as a list, Website Description as a string, TrustWorthiness Rating as a string, Visual Attractiveness Rating as a string"""
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": url
                    }
                },
                {
                    "type": "text",
                    "text": """Please determine whether you think the image of the website or Email is Phishing Image or Not.
                               If you believe the image is Phishy say "Something's Phishy", if not say "Nothing Phishy Here". 
                               If you believe the image is Phishy, then tell me what Phishy cues do you think are present in this list:
                               - Invalid Domain or Sender
                               - Poor Spelling or Grammar
                               - Appeal to Action - Greed 
                               - Appeal to Action - Urgency 
                               - Appeal to Action - Authority
                               - Potentially Malicious Link
                               - Other Phishy Findings
                               Then, tell me your opinion regarding the design and flow of the image. Using a five point Likert Scale, tell me how trustworthy you think the image:
                                - "Very Trustworthy"
                                - "Trustworthy"
                                - "Neutral" 
                                - "Untrustworthy" 
                                - "Very Untrustworthy".
                               What I am looking for here is for you to rate the level of trust you have that the contents of the e-mail or web page are real. Second, I want you to tell me how pleasing or attractive the website or e-mail looks.
                               In this case, your answer can range on a five-point Likert scale:
                                - “Very Visually Attractive or Appealing” 
                                - "Visually Attractive or Appealing"
                                - "Neutral"
                                - "Visually Unattractive or Unappealing"
                                - “Very Visually Unattractive or Unappealing”. 
                                If you don't like how an image looks, tell me.
                               """
                },
            ]
        }
    ]
    
    chat_completion = client.chat.completions.create(
    messages = Prompt,
    model= "2024-02-15-preview",
    max_tokens=3000
    )
    ans.loc[len(ans)]=[url, subject_id, maliciousness, chat_completion.choices[0].message.content]




In [ ]:
#ans
#ans.to_csv("ChatGPT_raw_output3new.csv", index=False)
#ans = pd.read_csv("ChatGPT_raw_output.csv")

In [ ]:
#Final = pd.DataFrame(columns= ["url", "subject_id", "if_malicious", "malicious_guess", "Present_Cue_Types_Guess", "Website_Description", "TrustWorthiness_Rating", "Visual_Attractiveness_Rating", "Comments"])
#Final = pd.read_csv("Save_Progress_GPT.csv")
#Final

Calcaluting ChatGPT stats of 1925 images

In [ ]:
import pandas as pd
Final = pd.read_csv("GPT_Answers_New_Prompt.csv")
Final

In [ ]:
Final = Final[Final['subject_id'].isin(unique_gold_subject_ids.tolist())]
Final

In [ ]:
Final = pd.merge(Final,gold_standard, on="subject_id")
Final

In [ ]:
# Cleaning data
Final.loc[Final['malicious_guess'] == 'Nothing Phishy Here', 'malicious_guess'] = "Nothing's Phishy Here"
Final['malicious_guess'].unique()

In [ ]:
hit_count = ((Final['if_malicious'] == True) & (Final['malicious_guess'] == "Something's Phishy")).sum()
Miss = ((Final['if_malicious'] == True) & (Final['malicious_guess'] == "Nothing's Phishy Here")).sum()
False_Alarm = ((Final['if_malicious'] == False) & (Final['malicious_guess'] == "Something's Phishy")).sum()
Correct_Rejection = ((Final['if_malicious'] == False) & (Final['malicious_guess'] == "Nothing's Phishy Here")).sum()

print(f"""Hit: {hit_count}
       Miss: {Miss}
       False_Alarm: {False_Alarm}
        Correct_Rejection: {Correct_Rejection}""")